In [2]:
import d4rl
import gym
import numpy as np

/home/users/adarsh/anaconda3/envs/diffuser/lib/python3.8/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
pybullet build time: May 20 2022 19:44:17


In [3]:
env = gym.make('maze2d-umaze-v1')
dataset = env.get_dataset()
np.where(dataset['timeouts']==1)

load datafile: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 33.64it/s]


(array([    91,    144,    230, ..., 999592, 999702, 999868]),)

In [4]:
env.unwrapped.goal_locations

[(1, 1)]

In [5]:
env.state_vector()

array([0.00099822, 0.00189032, 0.09982178, 0.18903173])

In [6]:
env.unwrapped.max_episode_steps = env._max_episode_steps
env.unwrapped.max_episode_steps

300

In [7]:
{k:v[0:10] for k,v in dataset.items()}

{'actions': array([[-0.56856084, -0.27724722],
        [-1.        ,  0.6666507 ],
        [-1.        ,  1.        ],
        [-0.57507885,  1.        ],
        [ 0.19998227,  0.3388329 ],
        [-0.85686654,  1.        ],
        [-0.70776117,  1.        ],
        [ 0.2815406 ,  1.        ],
        [ 0.3058027 ,  1.        ],
        [-0.17358942,  1.        ]], dtype=float32),
 'infos/goal': array([[3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871]], dtype=float32),
 'infos/qpos': array([[1.0856489, 1.9745734],
        [1.0843927, 1.97413  ],
        [1.0807577, 1.9752754],
        [1.0747498, 1.9787997],
        [1.0673866, 1.9846972],
        [1.0605173, 1.9913877],
        [1.0516236, 2.000444 ],
        [1.0410653, 2.0118601],
        [1.

In [8]:
{k:v[85:110] for k,v in dataset.items()}

{'actions': array([[-0.44485816, -0.28783458],
        [-1.        ,  0.10188948],
        [-0.86629534, -0.7061774 ],
        [-1.        , -0.43705118],
        [-0.83433855, -0.6813955 ],
        [-1.        ,  0.12431113],
        [-0.7744256 , -0.01577702],
        [-1.        , -0.85744804],
        [-0.42496255,  0.03549777],
        [-1.        , -0.16718516],
        [-0.9986516 ,  0.7941246 ],
        [-1.        , -1.        ],
        [-1.        , -0.46347594],
        [-0.47986785,  0.23541717],
        [-1.        ,  0.19184636],
        [-0.80567414,  0.72089   ],
        [-1.        ,  0.5853617 ],
        [-0.80314255, -0.38177225],
        [-0.5594637 , -0.3783816 ],
        [-0.8619271 ,  0.6341822 ],
        [-0.23198843, -0.05428014],
        [-0.8293132 ,  0.3993299 ],
        [-0.05379655,  0.1787489 ],
        [-1.        ,  0.14792313],
        [-0.92931616, -0.12765189]], dtype=float32),
 'infos/goal': array([[3.036284 , 3.022871 ],
        [3.036284 , 3.0228

In [10]:
n=15
print(len(dataset['actions']))
print(dataset.keys())
print(dataset['actions'][n])
print(dataset['observations'][n])
print(dataset['infos/qpos'][n])
print(dataset['infos/qvel'][n])

1000000
dict_keys(['actions', 'infos/goal', 'infos/qpos', 'infos/qvel', 'observations', 'rewards', 'terminals', 'timeouts'])
[-0.6284441   0.09111674]
[ 0.97895855  2.180523   -0.42206582  2.7247157 ]
[0.97895855 2.180523  ]
[-0.42206582  2.7247157 ]


In [ ]:
np.where(dataset['timeouts']==1)[0]

In [ ]:
{k:v[0:93] for k,v in dataset_naive.items()}

In [ ]:
import copy
from math import atan2, sin, cos, radians
dataset_naive = copy.deepcopy(dataset)
    

def calc_action(start, goal):
    delta_x = goal[0] - start[0]
    delta_y = goal[1] - start[1]
    theta_radians = atan2(delta_y, delta_x)
    a_x = cos(theta_radians)
    a_y = sin(theta_radians)
    return [a_x,a_y]
    

start = 0
for t in np.where(dataset['timeouts']==1)[0]:
    dataset_naive['actions'][start:t+1] = calc_action(dataset['infos/qpos'][start], dataset['infos/goal'][t])
    start = t+1
    
    

In [ ]:
dataset_naive['actions']

In [ ]:
dims = [1, 2, 4, 8]
list(zip(dims[:-1], dims[1:]))

In [ ]:
dims[1:]

In [55]:
def space_indices(num_steps, count):
    assert count <= num_steps

    if count <= 1:
        frac_stride = 1
    else:
        frac_stride = (num_steps - 1) / (count - 1)

    cur_idx = 0.0
    taken_steps = []
    for _ in range(count):
        taken_steps.append(round(cur_idx))
        cur_idx += frac_stride

    return taken_steps

In [64]:
steps = space_indices(100, 6)
log_count = min(len(steps) - 1, 10)
log_steps = [steps[i] for i in space_indices(len(steps) - 1, log_count)]
print(log_steps)
steps = steps[::-1]
pair_steps = zip(steps[:-1], steps[1:])

[0, 20, 40, 59, 79]


In [65]:
l = list(pair_steps)
l

[(99, 79), (79, 59), (59, 40), (40, 20), (20, 0)]

In [21]:
dataset['infos/qvel'].shape

(1000000, 2)

In [26]:
for x,y in dataset['infos/qvel']:
    print([x, y])
    break

[0.009810349, 0.02174424]


In [31]:
t = [[dataset['infos/qvel'][i][0],dataset['infos/qvel'][i][1]] for i in range(0,10)]

In [36]:
for i, (x,y) in enumerate(t):
    print(i, x, y)

0 0.009810349 0.02174424
1 -0.12562364 -0.044337805
2 -0.3634883 0.114539884
3 -0.60078645 0.35243094
4 -0.7363186 0.5897554
5 -0.6869364 0.66904855
6 -0.889375 0.90561897
7 -1.0558199 1.141626
8 -0.98625255 1.3770709
9 -0.9110725 1.611955
